In [1]:
import pandas as pd

from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import PreText, Select
from bokeh.plotting import figure

#output_notebook()

In [2]:
DEFAULT_TICKERS = ['FLOW', 'SPEED', 'DENSITY']
FILENAME = 'SP-280-Leste.csv'

def nix(val, lst):
    return [x for x in lst if x != val]

#@lru_cache()
def load_file(fname):
    data = pd.read_csv(fname, header=None, sep=',')
    
    # Drop the column '0'
    data = data.drop(columns=0)
    # Rename columns
    data = data.rename(columns={0:"ID", 1:"date",2:"FLOW", 3:"SPEED" })
    # Calculate flow rate
    data["FLOW"] = data["FLOW"] * 60 / 5 / 2 # two lanes
    # Calculate Density
    data["DENSITY"] = data["FLOW"] / data['SPEED']
    # Convert to_datetime
    data['date'] = pd.to_datetime(data['date'], format='%d/%m/%Y %H:%M')
    #Date-time column index
    data = data.set_index('date') 
    # Clean missing data
    data = data.fillna(0.0)
    
    return data
    

dataraw = load_file(FILENAME)

#@lru_cache()
def get_data(t1, t2):
    df1 = dataraw[t1]
    df2 = dataraw[t2]
    data = pd.concat([df1, df2], axis=1)
    
    return data

# set up widgets

stats = PreText(text='', width=500)
ticker1 = Select(value='FLOW', options=nix('FLOW', DEFAULT_TICKERS))
ticker2 = Select(value='SPEED', options=nix('SPEED', DEFAULT_TICKERS))
t1, t2 = ticker1.value, ticker2.value

# set up plots

source = ColumnDataSource(data=dict(date=[], FLOW=[], SPEED=[], DENSITY=[]))
source_static = ColumnDataSource(data=dict(date=[], FLOW=[], SPEED=[], DENSITY=[]))
tools = 'pan,wheel_zoom,xbox_select,reset'

corr = figure(plot_width=400, plot_height=400,
              tools='pan,wheel_zoom,box_select,reset')
corr.circle(t1, t2, size=2, source=source,
            selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)

ts1 = figure(plot_width=1500, plot_height=250, tools=tools, x_axis_type='datetime', active_drag="xbox_select")
ts1.line('date', t1, source=source_static)
ts1.circle('date', t1, size=1, source=source, color=None, selection_color="orange")

ts2 = figure(plot_width=1500, plot_height=250, tools=tools, x_axis_type='datetime', active_drag="xbox_select")
ts2.x_range = ts1.x_range
ts2.line('date', t2, source=source_static)
ts2.circle('date', t2, size=1, source=source, color=None, selection_color="orange")


# set up callbacks

def ticker1_change(attrname, old, new):
    ticker2.options = nix(new, DEFAULT_TICKERS)
    update()

def ticker2_change(attrname, old, new):
    ticker1.options = nix(new, DEFAULT_TICKERS)
    update()

def update(selected=None):
    t1, t2 = ticker1.value, ticker2.value

    data = dataraw.copy()
    source.data = source.from_df(dataraw)
    source_static.data = source.data
    update_stats(data)

    corr.title.text = '%s vs. %s' % (t1, t2)
    ts1.title.text, ts2.title.text = t1, t2

def update_stats(data):
    stats.text = str(data.describe())

ticker1.on_change('value', ticker1_change)
ticker2.on_change('value', ticker2_change)

def selection_change(attrname, old, new):
    t1, t2 = ticker1.value, ticker2.value
    data = dataraw.copy()
    selected = source.selected.indices
    if selected:
        data = data.iloc[selected, :]
    update_stats(data)

source.selected.on_change('indices', selection_change)

# set up layout
widgets = column(ticker1, ticker2, stats)
main_row = row(corr, widgets)
series = column(ts1, ts2)
layout = column(main_row, series)

# initialize
update()

curdoc().add_root(layout)
curdoc().title = "Traffic Time-based Analysis"

In [18]:
'''
DEFAULT_TICKERS = ['FLOW', 'SPEED', 'DENSITY']




def nix(val, lst):
    return [x for x in lst if x != val]

#@lru_cache()
def load_ticker(ticker, name):
    fname = 'SP-280-Leste.csv'
    data = pd.read_csv(fname, header=None, sep=',')
    
    # Drop the column '0'
    data = data.drop(columns=0)
    # Rename columns
    data = data.rename(columns={0:"ID", 1:"date",2:"FLOW", 3:"SPEED" })
    # Calculate flow rate
    data["FLOW"] = data["FLOW"] * 60 / 5 / 2 # two lanes
    # Calculate Density
    data["DENSITY"] = data["FLOW"] / data['SPEED']
    # Convert to_datetime
    data['date'] = pd.to_datetime(data['date'], format='%d/%m/%Y %H:%M')
    #Date-time column index
    data = data.set_index('date') 
    # Clean missing data
    data = data.fillna(0.0)
    
    return pd.DataFrame({name: data[ticker]})
    

#@lru_cache()
def get_data(t1, t2):
    df1 = load_ticker(t1, 't1')
    df2 = load_ticker(t2, 't2')
    data = pd.concat([df1, df2], axis=1)
    
    return data

# set up widgets

stats = PreText(text='', width=500)
ticker1 = Select(value='SPEED', options=nix('SPEED', DEFAULT_TICKERS))
ticker2 = Select(value='FLOW', options=nix('FLOW', DEFAULT_TICKERS))

# set up plots

source = ColumnDataSource(data=dict(date=[], t1=[], t2=[]))
source_static = ColumnDataSource(data=dict(date=[], t1=[], t2=[]))
tools = 'pan,wheel_zoom,xbox_select,reset'

corr = figure(plot_width=400, plot_height=400,
              tools='pan,wheel_zoom,box_select,reset')
corr.circle('t1', 't2', size=2, source=source,
            selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)

ts1 = figure(plot_width=1500, plot_height=250, tools=tools, x_axis_type='datetime', active_drag="xbox_select")
ts1.line('date', 't1', source=source_static)
ts1.circle('date', 't1', size=1, source=source, color=None, selection_color="orange")

ts2 = figure(plot_width=1500, plot_height=250, tools=tools, x_axis_type='datetime', active_drag="xbox_select")
ts2.x_range = ts1.x_range
ts2.line('date', 't2', source=source_static)
ts2.circle('date', 't2', size=1, source=source, color=None, selection_color="orange")

# set up callbacks

def ticker1_change(attrname, old, new):
    ticker2.options = nix(new, DEFAULT_TICKERS)
    update()

def ticker2_change(attrname, old, new):
    ticker1.options = nix(new, DEFAULT_TICKERS)
    update()

def update(selected=None):
    t1, t2 = ticker1.value, ticker2.value

    data = get_data(t1, t2)
    source.data = source.from_df(data)
    source_static.data = source.data

    update_stats(data, 't1', 't2')

    corr.title.text = '%s vs. %s' % (t1, t2)
    ts1.title.text, ts2.title.text = t1, t2

def update_stats(data, t1, t2):
    stats.text = str(data[[t1, t2]].describe())

ticker1.on_change('value', ticker1_change)
ticker2.on_change('value', ticker2_change)

def selection_change(attrname, old, new):
    t1, t2 = ticker1.value, ticker2.value
    data = get_data(t1, t2)
    selected = source.selected.indices
    if selected:
        data = data.iloc[selected, :]
    update_stats(data, t1, t2)

source.selected.on_change('indices', selection_change)

# set up layout
widgets = column(ticker1, ticker2, stats)
main_row = row(corr, widgets)
series = column(ts1, ts2)
layout = column(main_row, series)

# initialize
update()

curdoc().add_root(layout)
curdoc().title = "Traffic Time-based Analysis"

'''